In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode

In [ ]:
%%capture
%run "16. Eliminacion de outliers por categoría.ipynb"

In [3]:
# Criterio 2: Tiempos de respuesta mayores a tres desvíos estándar.

# Obtener todas las Columnas.
Columnas = list(dfs_Finales['Generales'].columns)

In [4]:
# IP Items sin asociar (tiempo).
IP_Items_Sin_Asociar_Tiempo = [Columna for Columna in Columnas if 'IP_Item' in Columna and '_Tiempo' in Columna and ('_Izq' not in Columna and '_Der' not in Columna)]

In [5]:
# IP Items asociados izquierda (tiempo).
IP_Items_Asociados_Izq_Tiempo = [Columna for Columna in Columnas if 'IP_Item' in Columna and '_Tiempo' in Columna and '_Izq' in Columna]

In [6]:
# IP Items asociados derecha (tiempo).
IP_Items_Asociados_Der_Tiempo = [Columna for Columna in Columnas if 'IP_Item' in Columna and '_Tiempo' in Columna and '_Der' in Columna]

In [ ]:
# Borrar filas en dónde el sujeto haya tardado más tiempo que 3 desvíos estándar.
for Nombre_df, df in dfs_Finales.items():
    #print(f"\n--- Procesando DataFrame: {Nombre_df} ---")
    
    dfs_Finales[Nombre_df] = f.Eliminar_Filas_Por_Desviacion_Estandar(
        df, 
        IP_Items_Sin_Asociar_Tiempo + IP_Items_Asociados_Izq_Tiempo + IP_Items_Asociados_Der_Tiempo, 
        3
    )
    
    #print(f"DataFrame {Nombre_df} procesado.")

In [11]:
# Aplanar y convertir a numérico.
Serie_Global = pd.Series(
    pd.to_numeric(
        dfs_Finales['Generales'][IP_Items_Sin_Asociar_Tiempo + IP_Items_Asociados_Izq_Tiempo + IP_Items_Asociados_Der_Tiempo].values.flatten(),
        errors = 'coerce'
    )
).dropna()

# Cálculo estadísticos.
Media_Global = 11.31
Desv_Estandar = 215.40
Umbral = 657.49

print(f"Media global: {Media_Global:.2f}, "
        f"Desvío estándar: {Desv_Estandar:.2f}, "
        f"Umbral: {Umbral:.2f}")

# Verificar atípicos.
Mascara_Atipicos = (
    dfs_Finales['Generales'][IP_Items_Sin_Asociar_Tiempo + IP_Items_Asociados_Izq_Tiempo + IP_Items_Asociados_Der_Tiempo] > Umbral
)
if Mascara_Atipicos.values.any():
    Total_Atipicos = int(Mascara_Atipicos.sum().sum())
    print(f"❌ {Total_Atipicos} valores de tiempo "
            "exceden el umbral.")
else:
    print("✅ No hay valores de tiempo atípicos.")

Media global: 11.31, Desvío estándar: 215.40, Umbral: 657.49
✅ No hay valores de tiempo atípicos.


In [9]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/17. Eliminacion de outliers por tiempos ({Nombre_df}).xlsx', index=False)